In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests 
import numpy as np

# to generate over 500 citites, install citypy (pip install citipy on gitbash terminal)

from citipy import citypy
# Api_key for the weather
from config import api_key
import os
import

ModuleNotFoundError: No module named 'citipy'